## Fetch Rewards Data Analyst Assessment

## Part 1: Review unstructured JSON data and diagram a new structured relational data model

### Our initial step is to convert the raw unstructurd JSON files into a more structured and readable format which then can be used to query. I will be using pandas library to prcoess & clean our data.


In [140]:
import gzip
import shutil
import os
import pandas as pd
from ast import literal_eval
import json
from datetime import datetime
from sqlalchemy import create_engine


#### Firstly, we will read the zipped json files through python.



In [141]:
for i in os.listdir():
  if 'json' in i:
    with gzip.open(i, 'rb') as f_in:
          with open(i.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [142]:
receipts = pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)

### Receipts Table

In [122]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


#### Before we proceed to the next step, we need to address a Data Quality issue right now before we finish cleaning the data further. We need to drop all the NaN values from the Receipts Table as shown below.

In [143]:
nan_df_re = receipts[receipts.isna().any(axis=1)]
nan_df_re.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
12,{'$oid': '5ff1e1b60a7214ada100055c'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687478000},{'$date': 1609687478000},NaN,{'$date': 1609687478000},NaN,8850.0,{'$date': 1612365878000},10.0,"[{'barcode': '034100573065', 'description': 'M...",FLAGGED,290.0,5ff1e194b6a9d73a3a9f1052
13,{'$oid': '5f9c74f70a7214ad07000037'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1604089079000},{'$date': 1604089079000},NaN,{'$date': 1609687494000},{'$date': 1604089080000},750.0,{'$date': 1604002679000},11.0,"[{'barcode': '075925306254', 'competitiveProdu...",REJECTED,1.0,5f9c74f7c88c1415cbddb839
15,{'$oid': '5ff1e1e90a7214ada1000569'},NaN,NaN,{'$date': 1609687529000},{'$date': 1609687529000},NaN,{'$date': 1609687529000},NaN,NaN,NaN,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff1e1e9b6a9d73a3a9f10f6
17,{'$oid': '5ff1e1b40a7214ada100055b'},750.0,"Receipt number 1 completed, bonus point schedu...",{'$date': 1609687476000},{'$date': 1609687476000},NaN,{'$date': 1609687477000},{'$date': 1609687476000},750.0,{'$date': 1609601076000},1.0,"[{'barcode': '075925306254', 'competitiveProdu...",FLAGGED,1.0,5ff1e1b4cfcf6c399c274a54


In [155]:
receipts = receipts.dropna(how='all')
receipts.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6


We can check the percentage % of missing values to quantify how clean the data is after droppping the NaN values.

In [156]:
percentage_null = receipts.isnull().mean()
for key,value in percentage_null.items():
    if value > 0:
        print(key,":",value*100)

bonusPointsEarned : 18.98116786343314
bonusPointsEarnedReason : 18.98116786343314
finishedDate : 19.11665086031703
pointsAwardedDate : 17.62633789459423
pointsEarned : 15.282482048502914
purchaseDate : 6.205121257282211
purchasedItemCount : 6.557377049180328
totalSpent : 5.893510364449261


### The 'rewardsReceiptItemList' field, which holds lists of receipts, must be exploded next (using the ast package's literal eval function). Then, using the pandas json normalize method, we must divide each dictionary apart and extract its columns. After that, we re-join this data to the data frame for receipts.


In [145]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [146]:

receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [147]:
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

In [148]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')


In [149]:
receipts_final


,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


### The additional fields that contain dictionaries need to be formatted now that "rewardsReceiptItemList" has been divided into numerous rows and the keys into various columns.



In [128]:
set().union(*(d.keys() for d in receipts_final['_id']))


{'$oid'}

In [129]:
set().union(*(d.keys() for d in receipts_final['createDate']))


{'$date'}

In [130]:
set().union(*(d.keys() for d in receipts_final['dateScanned']))


{'$date'}

In [131]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['finishedDate'])['finishedDate']))


{'$date'}

In [132]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['modifyDate'])['modifyDate']))


{'$date'}

In [133]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['pointsAwardedDate'])['pointsAwardedDate']))


{'$date'}

In [134]:
set().union(*(d.keys() for d in receipts_final.dropna(subset=['purchaseDate'])['purchaseDate']))


{'$date'}

### We just need to extract these fields because we can see that each of these columns only contains one dictionary item. The time is kept as UTC in the date columns, thus we use the apply function to change it to datetime format.

In [135]:
def date_converter(x):
  try:
    return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except TypeError:
    return(None)


In [136]:

receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])
receipts_final['createDate'] = receipts_final['createDate'].apply(lambda x: date_converter(x))
receipts_final['dateScanned'] = receipts_final['dateScanned'].apply(lambda x: date_converter(x))
receipts_final['finishedDate'] = receipts_final['finishedDate'].apply(lambda x: date_converter(x))
receipts_final['modifyDate'] = receipts_final['modifyDate'].apply(lambda x: date_converter(x))
receipts_final['pointsAwardedDate'] = receipts_final['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts_final['purchaseDate'] = receipts_final['purchaseDate'].apply(lambda x: date_converter(x))

In [137]:
receipts_final.head()


,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Users Table

In [21]:

users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [22]:
set().union(*(d.keys() for d in users['_id']))


{'$oid'}

In [23]:
set().union(*(d.keys() for d in users['createdDate']))


{'$date'}

In [24]:
set().union(*(d.keys() for d in users.dropna(subset=['lastLogin'])['lastLogin']))


{'$date'}

### We can see that each dictionary only has one key, therefore all that is left to do is extract the data using the previously established function to format the date in a way that is user-friendly.

In [25]:

users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [26]:
users.head()


,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


### Brands Table

In [90]:
brands.head()


,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [28]:
set().union(*(d.keys() for d in brands['cpg']))


{'$id', '$ref'}

### As there is more than one element in the dictionary, we need to explode the cpg fields.



In [29]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')

In [30]:
brands_norm

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [31]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')


In [32]:
brands_final.shape


(1167, 10)

In [33]:
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])


In [34]:
brands_final.head()


,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


# Part 2: Second: Write a query that directly answers a predetermined question from a business stakeholder

### To implement the SQL database for the queries, I'm utilizing SQLite using SQL Alchemy Python.

In [69]:
engine = create_engine('sqlite://', echo=False)
     

users.drop_duplicates(subset=['_id']).to_sql('users',con=engine)
     

receipts_final.columns = receipts_final.columns.str.replace(".", "_")
     

receipts_final.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=engine)
     

brands_final.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands',con=engine)

C:\Users\nsuman2\AppData\Local\Temp\ipykernel_35384\4097467866.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  receipts_final.columns = receipts_final.columns.str.replace(".", "_")


898

### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

I'll presume that the terms "Finished" and "Accepted" are interchangeable.

The average spend from receipts with an accepted "rewardsReceiptStatus" is higher than the average spend from receipts with a rejected "rewardsReceiptStatus."

In [73]:
engine.execute("SELECT r.rewardsReceiptStatus, avg(r.totalSpent) FROM receipts r GROUP BY r.rewardsReceiptStatus;").fetchall()

[('FINISHED', 1244.372934121628),
 ('FLAGGED', 2635.5702469136068),
 ('PENDING', 28.03244897959184),
 ('REJECTED', 19.544970059880253),
 ('SUBMITTED', None)]

Receipts with "rewardsReceiptStatus" of accepted had more purchases than receipts with "rewardsReceiptStatus" of rejected, assuming that accepted is synonymous with finished.

In [74]:
engine.execute("SELECT r.rewardsReceiptStatus, sum(r.purchasedItemCount) FROM receipts r GROUP BY r.rewardsReceiptStatus;").fetchall()

[('FINISHED', 1364998.0),
 ('FLAGGED', 294816.0),
 ('PENDING', None),
 ('REJECTED', 740.0),
 ('SUBMITTED', None)]


###  Let us first find the latest created date 

The latest created date is from February. The Output is as below - 

In [75]:
engine.execute("SELECT createdDate from users ORDER BY createdDate desc LIMIT 10;").fetchall()

[('2021-02-12 14:11:06',),
 ('2021-02-12 14:10:49',),
 ('2021-02-11 16:17:53',),
 ('2021-02-11 16:17:49',),
 ('2021-02-11 16:17:07',),
 ('2021-02-11 14:01:17',),
 ('2021-02-11 14:00:47',),
 ('2021-02-10 19:53:01',),
 ('2021-02-09 14:17:52',),
 ('2021-02-09 14:17:41',)]

#### Which brand has the most spend among users who were created within the past 6 months?

In [93]:
engine.execute("SELECT * from (SELECT r.rewardsReceiptItemList_brandCode,SUM(r.totalSpent) as SummedTotalSpent from receipts r where r.userId in (SELECT _id from users u where u.createdDate < (SELECT strftime('%Y-%m-%d %H:%M:%S', 'now','-6 months')))group by r.rewardsReceiptItemList_brandCode) where rewardsReceiptItemList_brandCode is not NULL order by SummedTotalSpent DESC LIMIT 1;").fetchall()


[('BEN AND JERRYS', 198089.38000000027)]

####  Which brand has the most transactions among users who were created within the past 6 months?

In [94]:
engine.execute("SELECT * from (SELECT r.rewardsReceiptItemList_brandCode,COUNT(r.totalSpent) as MostTransactions from receipts r where r.userId in (SELECT _id from users u where u.createdDate < (SELECT strftime('%Y-%m-%d %H:%M:%S', 'now','-6 months')))group by r.rewardsReceiptItemList_brandCode) where rewardsReceiptItemList_brandCode is not NULL order by MostTransactions DESC LIMIT 1;").fetchall()


[('HY-VEE', 291)]

## Part 3: Evaluate Data Quality Issues


### To identify data quality problems I made the decision to investigate the brandCode mapping between the brands data and the receipts data.


In [99]:
len(set(receipts_final.dropna(subset=['rewardsReceiptItemList_brandCode'])['rewardsReceiptItemList_brandCode']))


227

In [100]:
len(set(brands_final.dropna(subset=['brandCode'])['brandCode']))


897

#### As you can see, the brands dataset contains a lot more distinct brand codes than the receipts collection. We now contrast the two to determine whether the brand codes on all of the receipts can be mapped to the brands table.

In [101]:
receipts_dropped = receipts_final.dropna(subset=['rewardsReceiptItemList_brandCode'])
receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))

C:\Users\nsuman2\AppData\Local\Temp\ipykernel_35384\2750043545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receipts_dropped['rewardsReceiptItemList_brandCode'] = receipts_dropped['rewardsReceiptItemList_brandCode'].apply(lambda x:str(x))


In [102]:

brands_final['barcode'] = brands_final['barcode'].apply(lambda x: str(x))
brands_final['brandCode'] = brands_final['brandCode'].apply(lambda x: str(x))
     

In [103]:
receipts_with_brands = pd.merge(receipts_final,brands_final,left_on='rewardsReceiptItemList_brandCode',right_on='brandCode')


#### Every receipt in the following image has a brand code, yet the codes are absent from the brands table. When we query between the two tables, a lot of data will be MISSING, which would be an issue.

In [104]:
set(receipts_dropped[~(receipts_dropped['rewardsReceiptItemList_brandCode'].isin(brands['brandCode']))]['rewardsReceiptItemList_brandCode'])


{'7UP',
 'ADVIL',
 'AMERICAN BEAUTY',
 'ARROWHEAD',
 'AZTECA',
 'BANZA',
 'BEAR CREEK COUNTRY KITCHENS',
 'BEN AND JERRYS',
 'BETTY CROCKER',
 'BIC',
 'BIGELOW',
 'BLUE DIAMOND',
 "BOAR'S HEAD",
 'BORDEN',
 'BOTA BOX',
 'BRAND',
 "BRASWELL'S",
 'BUNNY',
 "BUSH'S BEST",
 'C&H',
 'CADBURY',
 'CAL-ORGANIC FARMS',
 'CALIFIA FARMS',
 "CAMPBELL'S",
 'CARAMELLO',
 'CHEERIOS',
 'CHEESE',
 'CHEEZ-IT',
 'CHEX',
 'CHICKEN OF THE SEA',
 'CHIQUITA',
 'CINNAMON TOAST CRUNCH',
 'COKE',
 'COLEMAN NATURAL',
 "CONNIE'S PIZZA",
 'CREST 3D WHITE',
 'CRISPIX',
 'DANNON',
 'DARE',
 'DELI',
 'DIET COKE',
 'DIGIORNO',
 'DOLE',
 'DR PEPPER',
 'EDWARDS',
 "EGGLAND'S BEST",
 'EGGO',
 'EL MONTEREY',
 'ENERGIZER MAX',
 'ESSENTIAL EVERYDAY',
 'FAGE',
 "FAMOUS DAVE'S",
 "FLORIDA'S NATURAL",
 'FOLGERS',
 'FORTUNE YAKISOBA',
 'FRANZ',
 "FRENCH'S",
 'FRESH EXPRESS',
 'FRESH STEP',
 'FRONTERA',
 'GALLO FAMILY VINEYARDS',
 'GENERAL MILLS',
 'GERBER',
 'GERM-X',
 'GREEN GIANT',
 'GRIMMWAY FARMS',
 'HANOVER',
 'HARVEST SNA

## Data Quality issue 2 -  Clean up NaN Values

In [106]:
nan_df = brands_final[brands_final.isna().any(axis=1)]
nan_df.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '559c2234e4b0...",J.L. Kraft,NaN,J.L. KRAFT,Cogs,559c2234e4b06aca36af13c6
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5a734034e4b0...",Campbell's Home Style,0.0,CAMPBELLS HOME STYLE,Cogs,5a734034e4b0d58f376be874
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '59ba6f1ce4b0...",test,NaN,TEST,Cogs,59ba6f1ce4b092b29c167346
10,5f4bf556be37ce0b4491554d,511111516354,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f4bf556be37...",test brand @1598813526777,NaN,TEST BRANDCODE @1598813526777,Cogs,5f4bf556be37ce0b44915549
11,57c08106e4b0718ff5fcb02c,511111102540,NaN,NaN,"{'$ref': 'Cpgs', '$id': {'$oid': '5332f5f2e4b0...",MorningStar,NaN,nan,Cpgs,5332f5f2e4b03c9a25efd0aa


In [108]:
brands_final = brands_final.dropna(how='all')
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,nan,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


### Quantifying Missing Data

In [152]:
receipts_final.isnull()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,False,False,True,False,True,False,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,False,False,True,True,False,False,True,False,True,True,...,True,True,True,True,True,True,True,True,True,True
7377,False,False,True,True,False,False,True,False,True,True,...,True,True,True,True,True,True,True,True,True,True
7378,False,False,False,False,False,False,True,False,True,False,...,True,True,True,True,True,True,True,True,False,True
7379,False,False,False,False,False,False,True,False,True,False,...,True,True,True,True,True,True,True,True,False,True


In [161]:
brands_final.isnull()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1162,False,False,False,False,False,False,True,False,False,False
1163,False,False,False,True,False,False,True,False,False,False
1164,False,False,False,False,False,False,True,False,False,False
1165,False,False,False,True,False,False,False,False,False,False


In [163]:
brands_final.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode         0
cpg.$ref          0
cpg.$id.$oid      0
dtype: int64

In [162]:
users.isnull()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
490,False,False,False,False,False,True,True
491,False,False,False,False,False,True,True
492,False,False,False,False,False,True,True
493,False,False,False,False,False,True,True


In [164]:
users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

## By quantifying the sum or % of missing data, we can get a clear idea on how to proceed with data exploration and data cleaning in order to maintain data quality throughout the process.